In [ ]:
import pandas as pd
all_pr_df = pd.read_parquet("hf://datasets/hao-li/AIDev/all_pull_request.parquet")
all_repo_df = pd.read_parquet("hf://datasets/hao-li/AIDev/all_repository.parquet")
all_user_df = pd.read_parquet("hf://datasets/hao-li/AIDev/all_user.parquet")

In [ ]:

# Ensure created_at is datetime
all_pr_df['created_at'] = pd.to_datetime(all_pr_df['created_at'])

# Count agent PRs per repo
pr_counts = all_pr_df.groupby('repo_id').size().reset_index(name='agent_pr_count')

# Filter for repos with >= 1000 agent PRs
large_repos = pr_counts[pr_counts['agent_pr_count'] >= 100]

# Get first agent PR date per repo
first_pr_dates = all_pr_df.groupby('repo_id')['created_at'].min().reset_index(name='first_pr_date')

# Merge counts and first date
result = large_repos.merge(first_pr_dates, on='repo_id')

# Merge with repo metadata for repo name and URL
result = result.merge(all_repo_df, left_on='repo_id', right_on='id')

# Select relevant columns
result = result[['full_name', 'url', 'agent_pr_count', 'first_pr_date']]

# Sort by number of agent PRs descending
result = result.sort_values(by='agent_pr_count', ascending=False)

# Take top 100 repos


# Save to TXT
with open("top_agent_repos.txt", "w", encoding="utf-8") as f:
    for idx, row in result.iterrows():
        f.write(f"{row['full_name']} | {row['url']} | {row['agent_pr_count']} agent PRs | first PR: {row['first_pr_date'].date()}\n")

print("Saved top 100 agent repos to top_agent_repos.txt")

In [ ]:
from github import Github, Auth
import pandas as pd
from datetime import datetime
import time

# --- CONFIGURATION ---
GITHUB_TOKEN = ""  
INPUT_FILE = "repos_min1000pr.txt"
OUTPUT_FILE = "filtered_repos.txt"
MIN_YEARS = 3
RATE_LIMIT_DELAY = 0.5  # seconds between API calls

# --- Initialize GitHub ---
g = Github(auth=Auth.Token(GITHUB_TOKEN))

# --- Read TXT file ---
repos = []
with open(INPUT_FILE, "r", encoding="utf-8") as f:
    for line in f:
        parts = line.strip().split("|")
        if len(parts) < 4:
            continue  # skip malformed lines
        full_name = parts[0].strip()
        repo_url = parts[1].strip()
        try:
            agent_pr_count = int(parts[2].strip().split()[0])
        except:
            agent_pr_count = 0
        # Extract date after "first PR:"
        first_agent_pr_str = parts[3].strip()
        if first_agent_pr_str.lower().startswith("first pr:"):
            first_agent_pr_str = first_agent_pr_str.split(":", 1)[1].strip()
        try:
            first_agent_pr = datetime.fromisoformat(first_agent_pr_str).replace(tzinfo=pd.Timestamp.utcnow().tz)
        except:
            first_agent_pr = pd.NaT
        repos.append((full_name, repo_url, agent_pr_count, first_agent_pr))

df = pd.DataFrame(repos, columns=["full_name", "repo_url", "agent_pr_count", "first_agent_pr"])

# --- Fetch GitHub repo creation dates ---
creation_dates = []
for idx, row in df.iterrows():
    try:
        repo = g.get_repo(row['full_name'])
        creation_dates.append(repo.created_at)  # timezone-aware
        time.sleep(RATE_LIMIT_DELAY)
    except Exception as e:
        print(f"Failed to get {row['full_name']}: {e}")
        creation_dates.append(pd.NaT)

df['repo_creation_date'] = creation_dates

# --- Filter for >=3 years history ---
three_years = pd.Timedelta(days=MIN_YEARS*365)
df_filtered = df.dropna(subset=['repo_creation_date', 'first_agent_pr'])
df_filtered = df_filtered[df_filtered['first_agent_pr'] - df_filtered['repo_creation_date'] >= three_years]

# --- Save filtered repos ---
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    for _, row in df_filtered.iterrows():
        f.write(
            f"{row['full_name']} | {row['repo_url']} | {row['agent_pr_count']} agent PRs | "
            f"first agent PR: {row['first_agent_pr'].date()} | repo creation: {row['repo_creation_date'].date()}\n"
        )

print(f"Filtered repos saved to {OUTPUT_FILE}")
